
# AWSDX Single Server > sc24-demo 
#### (derived from) Functional Test 3.2.1 - Local bridge with Shared NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.2.1 for testing Shared NIC with a local bridge.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [28]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,mcevik_0000100641
Bastion Private Key File,/home/fabric/work/fabric_config/mcevik_0000100641_bastion
Slice Private Key File,/home/fabric/work/fabric_config/mcevik_0000100641_sliver
Slice Public Key File,/home/fabric/work/fabric_config/mcevik_0000100641_sliver.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,mcevik_0000100641
Bastion Private Key File,/home/fabric/work/fabric_config/mcevik_0000100641_bastion
Slice Private Key File,/home/fabric/work/fabric_config/mcevik_0000100641_sliver
Slice Public Key File,/home/fabric/work/fabric_config/mcevik_0000100641_sliver.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [29]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   Xilinx Tools Storage Slice
Slice ID     e873c70a-6858-488a-bed9-1dfd5b13493b
Slice State  StableOK
Lease End    2025-04-23 22:27:24 +0000
-----------  ------------------------------------
-----------  ------------------------------------
Slice Name   FIU - reserve worker-3 spare
Slice ID     98a2459b-1c6b-4577-ba61-4137553e5838
Slice State  AllocatedOK
Lease End    2025-04-17 03:00:00 +0000
-----------  ------------------------------------


## Step 4: Create the Slice

The following creates two nodes with a shared NIC each. This should be run all worker nodes regardless of type.

Two nodes with one NIC component each are created on different workers.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. 

**Be sure to try different combinations of workers**

In [30]:
from datetime import datetime
from dateutil import tz

name1='Node1'
nic1_name='SharedNIC1'

name2='Node2'
nic2_name='SharedNIC2'

network_name='l2-bridge'

site='FIU'

# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
#worker1=f'{site.lower()}-w1.fabric-testbed.net'
#worker2=f'{site.lower()}-w2.fabric-testbed.net'

cores=10
ram=20
disk=50

slice_name=f"AWSDX sc2024-demo {site} {datetime.now().strftime('%Y-%m-%d-%H-%M')}"

### List of the VM images

Available images on FABRIC Testbed

```
+------------------------------+
| Name                         |
+------------------------------+
| default_debian_11            |
| default_debian_12            |
| default_rocky_8              |
| default_rocky_9              |
| default_ubuntu_20            |
| default_ubuntu_22            |
| default_ubuntu_24            |
| docker_rocky_8               |
| docker_rocky_9               |
| docker_ubuntu_20             |
| docker_ubuntu_22             |
| docker_ubuntu_24             |
+------------------------------+
```

In [31]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=name1, site=site, cores=cores, ram=ram, disk=disk, image="docker_rocky_8")
    iface1 = node1.add_component(model='NIC_Basic', name=nic1_name).get_interfaces()[0]
    
    node2 = slice.add_node(name=name2, site=site, cores=cores, ram=ram, disk=disk, image="default_debian_12")
    iface2 = node2.add_component(model='NIC_Basic', name=nic2_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])
 
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 10, Time: 238 sec


ID,ea9155c7-7d1d-418c-82c6-c966493f8c34
Name,AWSDX sc2024-demo FIU 2025-04-14-21-25
Lease Expiration (UTC),2025-04-15 21:25:48 +0000
Lease Start (UTC),2025-04-14 21:25:48 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
335bf561-b93f-4754-9b69-dc8b185314d3,Node1,10,32,100,docker_rocky_8,qcow2,fiu-w4.fabric-testbed.net,FIU,rocky,131.94.57.51,Active,,ssh -i /home/fabric/work/fabric_config/mcevik_0000100641_sliver -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.51,/home/fabric/work/fabric_config/mcevik_0000100641_sliver.pub,/home/fabric/work/fabric_config/mcevik_0000100641_sliver
9b76dbc8-f2cf-457e-ae5f-a6655e40dfa5,Node2,10,32,100,default_debian_12,qcow2,fiu-w4.fabric-testbed.net,FIU,debian,131.94.57.58,Active,,ssh -i /home/fabric/work/fabric_config/mcevik_0000100641_sliver -F /home/fabric/work/fabric_config/ssh_config debian@131.94.57.58,/home/fabric/work/fabric_config/mcevik_0000100641_sliver.pub,/home/fabric/work/fabric_config/mcevik_0000100641_sliver


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e96edb9e-f329-4fcd-a88a-ac694a990701,l2-bridge,L2,L2Bridge,FIU,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-SharedNIC1-p1,p1,Node1,l2-bridge,100,config,,06:8D:B4:9B:91:AF,enp7s0,enp7s0,None,4,HundredGigE0/0/0/11
Node2-SharedNIC2-p1,p1,Node2,l2-bridge,100,config,,0A:FB:6A:2F:EC:A2,enp7s0,enp7s0,fe80::8fb:6aff:fe2f:eca2,4,HundredGigE0/0/0/11



Time to print interfaces 242 seconds


## Step 5: Observe the Slice's Attributes

### Print the slice 

In [32]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  --------------------------------------
Slice Name   AWSDX sc2024-demo FIU 2025-04-14-21-25
Slice ID     ea9155c7-7d1d-418c-82c6-c966493f8c34
Slice State  StableOK
Lease End    2025-04-15 21:25:48 +0000
-----------  --------------------------------------


## Print the Node List

In [33]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
335bf561-b93f-4754-9b69-dc8b185314d3,Node1,10,32,100,docker_rocky_8,qcow2,fiu-w4.fabric-testbed.net,FIU,rocky,131.94.57.51,Active,,ssh -i /home/fabric/work/fabric_config/mcevik_0000100641_sliver -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.51,/home/fabric/work/fabric_config/mcevik_0000100641_sliver.pub,/home/fabric/work/fabric_config/mcevik_0000100641_sliver
9b76dbc8-f2cf-457e-ae5f-a6655e40dfa5,Node2,10,32,100,default_debian_12,qcow2,fiu-w4.fabric-testbed.net,FIU,debian,131.94.57.58,Active,,ssh -i /home/fabric/work/fabric_config/mcevik_0000100641_sliver -F /home/fabric/work/fabric_config/ssh_config debian@131.94.57.58,/home/fabric/work/fabric_config/mcevik_0000100641_sliver.pub,/home/fabric/work/fabric_config/mcevik_0000100641_sliver


## Print the Node Details

In [34]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  --------------------------------------------------------------------------------------------------------------------------------
ID                 335bf561-b93f-4754-9b69-dc8b185314d3
Name               Node1
Cores              10
RAM                32
Disk               100
Image              docker_rocky_8
Image Type         qcow2
Host               fiu-w4.fabric-testbed.net
Site               FIU
Management IP      131.94.57.51
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/mcevik_0000100641_sliver -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.51
-----------------  --------------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------------------
ID                 9b76dbc8-f2cf-457e-ae5f-a6655e40dfa5
Name

## Print the Interfaces

You should see 2 interfaces.

In [35]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-SharedNIC1-p1,p1,Node1,l2-bridge,100,config,,06:8D:B4:9B:91:AF,enp7s0,enp7s0,None,4,HundredGigE0/0/0/11
Node2-SharedNIC2-p1,p1,Node2,l2-bridge,100,config,,0A:FB:6A:2F:EC:A2,enp7s0,enp7s0,fe80::8fb:6aff:fe2f:eca2,4,HundredGigE0/0/0/11


## Step 6: Configure interfaces, test reachability

##  Configure IP Addresses

### Pick a Subnet

Create a subnet and list of available IP addresses. You can use either IPv4 or IPv6 subnets and addresses.

In [36]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [37]:
try:
    node1 = slice.get_node(name=name1)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:8d:b4:9b:91:af brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global enp7s0
       valid_lft forever preferred_lft forever
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:8d:b4:9b:91:af brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global enp7s0
       valid_lft forever preferred_lft forever



### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [38]:
try:
    node2 = slice.get_node(name=name2)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:fb:6a:2f:ec:a2 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::8fb:6aff:fe2f:eca2/64 scope link 
       valid_lft forever preferred_lft forever
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:fb:6a:2f:ec:a2 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::8fb:6aff:fe2f:eca2/64 scope link 
       valid_lft forever preferred_lft forever



### Test reachability

Test ping between interfaces, observe successful output.


In [39]:
try:
    node1 = slice.get_node(name=name1)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=64 time=3.62 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=64 time=0.116 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=64 time=0.092 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=64 time=0.091 ms
64 bytes from 192.168.1.2: icmp_seq=5 ttl=64 time=0.098 ms

--- 192.168.1.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4061ms
rtt min/avg/max/mdev = 0.091/0.804/3.624/1.410 ms
PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=64 time=3.62 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=64 time=0.116 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=64 time=0.092 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=64 time=0.091 ms
64 bytes from 192.168.1.2: icmp_seq=5 ttl=64 time=0.098 ms

--- 192.168.1.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4061ms
rtt min/avg/max/mdev = 0.091/0.804/3.624/1.410 ms




## Step 7: Execute sc24-demo

- https://sdx-docs.readthedocs.io/en/latest/sdx_deploy_single_server.html#


### Select the node 

- node1 : created with the docker_rocky_8 image that already has docker installed. No further action needed for docker installation
- node2 : created with the default_debian_12 image.  Install docker -> https://docs.docker.com/engine/install/debian/


In [42]:

node = node1
#node = node2

if node == node1:
    print(f'--- Use Node: {node}')
    commands = list()
    commands.append(f'sudo dnf install -y jq')

elif node == node2: 
    print(f'--- Use Node: {node}')
    commands = list()
    commands.append(f'sudo apt-get update')
    commands.append(f'sudo apt-get -y install ca-certificates curl')
    commands.append(f'sudo install -m 0755 -d /etc/apt/keyrings')
    commands.append(f'sudo curl -fsSL https://download.docker.com/linux/debian/gpg -o /etc/apt/keyrings/docker.asc')
    commands.append(f'sudo chmod a+r /etc/apt/keyrings/docker.asc')
    commands.append(f'echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/debian $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null')
    commands.append(f'sudo apt-get update')
    commands.append(f'sudo apt-get -y install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin')
    commands.append(f'sudo groupadd docker')
    commands.append(f'sudo usermod -aG docker $USER')
    commands.append(f'sudo apt-get install -y jq')


for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


--- Use Node: -----------------  --------------------------------------------------------------------------------------------------------------------------------
ID                 335bf561-b93f-4754-9b69-dc8b185314d3
Name               Node1
Cores              10
RAM                32
Disk               100
Image              docker_rocky_8
Image Type         qcow2
Host               fiu-w4.fabric-testbed.net
Site               FIU
Management IP      131.94.57.51
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/mcevik_0000100641_sliver -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.51
-----------------  --------------------------------------------------------------------------------------------------------------------------------
Executing sudo dnf install -y jq
Last metadata expiration check: 0:03:09 ago on Mon Apr 14 21:42:51 2025.
Package jq-1.6-9.el8_10.x86_64 is already installed.
Dependencies resolved.
Nothing to do

#### Check Management IP address (IPv4, IPv6)

In [43]:
import ipaddress

def check_ip_version(ip_address):
    try:
        ip_object = ipaddress.ip_address(ip_address)
        if isinstance(ip_object, ipaddress.IPv4Address):
            return "IPv4"
        elif isinstance(ip_object, ipaddress.IPv6Address):
            return "IPv6"
    except ValueError:
        return "Invalid IP address"


### 1. Setup


<div class="alert alert-block alert-info">
<b>Testing - Declarations > </b> SC24 Demo
</div>

In [70]:

sdx_work_repo = 'https://github.com/atlanticwave-sdx/sdx-sc24-demo.git'
sdx_work_branch = 'main'
repo_dir = 'awsdx/sdx-sc24-demo'
docker_compose_dir = repo_dir


### 2. Checkout

In [53]:
# Testing - checkout

commands1 = list()
commands1.append(f'[ ! -d ~/{repo_dir} ] && mkdir -p ~/{repo_dir}')
commands1.append(f'cd ~/{repo_dir} && git init && git remote add origin {sdx_work_repo}')
commands1.append(f'cd ~/{repo_dir} && git fetch origin')
commands1.append(f'cd ~/{repo_dir} && git checkout {sdx_work_branch}')


#
# Commenting out the IPv6 customization to re-use when needed
#

# Testing - customize for IPv6

###########   INFORMATIVE  - Begin  ############
# Add the following block to the docker-compose.yml
# Ref: https://docs.docker.com/reference/compose-file/networks/#the-default-network
#
#networks:
#   default:
#     name: sdxnet
#     enable_ipv6: true
#     ipam:
#       config:
#         - subnet: 2001:db8::/64
###########   INFORMATIVE   - End  ############

# Comment out
#management_ip = node.get_management_ip()

#if check_ip_version(management_ip) == 'IPv6':
#    commands1.append(f'cd ~/{repo_dir}/data-plane && echo -e "networks:\n  default:\n    name: sdxnet\n    enable_ipv6: true\n    ipam:\n      config:\n        - subnet: 2001:db8::/64" | tee -a docker-compose.yml > /dev/null')


for command in commands1:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

Executing [ ! -d ~/awsdx/sdx-sc24-demo ] && mkdir -p ~/awsdx/sdx-sc24-demo
Executing cd ~/awsdx/sdx-sc24-demo && git init && git remote add origin https://github.com/atlanticwave-sdx/sdx-sc24-demo.git
Initialized empty Git repository in /home/rocky/awsdx/sdx-sc24-demo/.git/
 hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
 Executing cd ~/awsdx/sdx-sc24-demo && git fetch origin
 From https://github.com/atlanticwave-sdx/sdx-sc24-demo
 * [new branch]      main                   -> origin/main
 * [new branch]      mininet-service-update -> origin/mininet-servic

### 3. Start

##### ***SC24-Demo***


In [54]:

commands2 = list()
commands2.append(f'sudo systemctl start docker')
commands2.append(f'cd ~/{repo_dir} && docker compose up -d &> /tmp/docker-compose.log')
commands2.append(f'cd ~/{repo_dir} && docker compose ps -a')


for command in commands1 + commands2:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


Executing [ ! -d ~/awsdx/sdx-sc24-demo ] && mkdir -p ~/awsdx/sdx-sc24-demo
Executing cd ~/awsdx/sdx-sc24-demo && git init && git remote add origin https://github.com/atlanticwave-sdx/sdx-sc24-demo.git
Reinitialized existing Git repository in /home/rocky/awsdx/sdx-sc24-demo/.git/
 error: remote origin already exists.
 Executing cd ~/awsdx/sdx-sc24-demo && git fetch origin
Executing cd ~/awsdx/sdx-sc24-demo && git checkout mininet-service-update
Your branch is up to date with 'origin/mininet-service-update'.
 Already on 'mininet-service-update'
 Executing sudo systemctl start docker
Executing cd ~/awsdx/sdx-sc24-demo && docker compose up -d &> /tmp/docker-compose.log
Executing cd ~/awsdx/sdx-sc24-demo && docker compose ps -a
NAME                              IMAGE                         COMMAND                  SERVICE           CREATED          STATUS                   PORTS
sdx-sc24-demo-amlight-1           amlight/kytos-sdx:latest      "/bin/bash -x -c '# …"   amlight           9 sec

### 4. Stop (Optional)

In [55]:

commands = list()
commands.append(f'cd ~/{docker_compose_dir} && docker compose down')


for command in commands:
    print(f'Executing {command}')
    ###stdout, stderr = node.execute(command)
    
print('Done')


Executing cd ~/awsdx/sdx-sc24-demo && docker compose down
Done


### 5. Prune system  (Optional)

##### 5.1 Prune docker

In [56]:

commands = list()
commands.append(f'cd ~/{docker_compose_dir} && docker system prune -a -f --volumes')


for command in commands:
    print(f'Executing {command}')
    ###stdout, stderr = node.execute(command)
    
print('Done')


Executing cd ~/awsdx/sdx-sc24-demo && docker system prune -a -f --volumes
Done


##### 5.2 Prune directory

In [57]:

commands = list()
commands.append(f'rm -rf ~/{repo_dir}')


for command in commands:
    print(f'Executing {command}')
    ###stdout, stderr = node.execute(command)
    
print('Done')


Executing rm -rf ~/awsdx/sdx-sc24-demo
Done


### 6. Execute Tests

##### 6.1 Set locations

In [64]:

remote_directory_path = repo_dir
script_dir = '~/' + remote_directory_path + '/scripts'


##### 6.2 Create L2VPN

In [68]:
stdout, stderr = node.execute(f'cd {remote_directory_path} && ./scripts/create-l2vpn.sh')

{
  "reason": "Connection published",
  "service_id": "156a0c7e-d92e-4d1f-a451-ef97f5890d33",
  "status": "UNDER_PROVISIONING"
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   315  100   129  100   186    941   1357 --:--:-- --:--:-- --:--:--  2299
 

##### 6.3 Test L2VPN

In [69]:
service_id = '156a0c7e-d92e-4d1f-a451-ef97f5890d33'
stdout, stderr = node.execute(f'cd {remote_directory_path} && script -c "./scripts/test-l2vpn.sh {service_id}" /dev/null')

Script started, file is /dev/null
Configuring hostA (geantFRH1)...
Configuring hostZ (tenetH1)...
Learning MAC addresses...
ping test HostA - HostB
+ docker exec -it 6e09a27dcc43a2d71be95f7391bf6b062eeb55c572bfac3f05846165a99df021 bash -c 'mnexec -a 170 ping6 -c5 -i0.2 2001:db8:4092:4093::2'
PING 2001:db8:4092:4093::2(2001:db8:4092:4093::2) 56 data bytes
64 bytes from 2001:db8:4092:4093::2: icmp_seq=1 ttl=64 time=0.169 ms
64 bytes from 2001:db8:4092:4093::2: icmp_seq=2 ttl=64 time=0.075 ms
64 bytes from 2001:db8:4092:4093::2: icmp_seq=3 ttl=64 time=0.068 ms
64 bytes from 2001:db8:4092:4093::2: icmp_seq=4 ttl=64 time=0.033 ms
64 bytes from 2001:db8:4092:4093::2: icmp_seq=5 ttl=64 time=0.034 ms

--- 2001:db8:4092:4093::2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 830ms
rtt min/avg/max/mdev = 0.033/0.075/0.169/0.049 ms
Script done, file is /dev/null


### Docker Hub Login (optional)
Create personal access token -> https://docs.docker.com/security/for-developers/access-tokens/

In [ ]:
# Docker Login - when necessary

docker_access_token = '<DOCKER_PERSONAL_ACCESS_TOKEN>'
docker_user_account = '<DOCKER_USER_ACCOUNT>'

commands = list()
#commands.append(f'echo {docker_access_token} | docker login --username {docker_user_account} --password-stdin')

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

## Step 8: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    ###slice.delete()
except Exception as e:
    print(f"Exception: {e}")